# MIMIC3 Data Preparation

Steps:

1) Download MIMIC-III dataset
2) Preprocess data using the code provided in https://github.com/YerevaNN/mimic3-benchmarks. Only step 1 to 4 are used.

In [2]:
import numpy as np
import pandas as pd
import os
from glob import glob

In [ ]:
# path to all_diagnoses and all_stays from original MIMIC-III dataset from step 1
all_diagnoses = pd.read_csv('mimic3-benchmarks/data/root/all_diagnoses.csv')
all_stays = pd.read_csv('mimic3-benchmarks/data/root/all_stays.csv')

# path to preprocessed data from step 2
dir = 'mimic3-benchmarks/data/root'

In [ ]:
events_files = [y for x in os.walk(dir) for y in glob(os.path.join(x[0], 'events.csv'))]

# select interested item_id from D_LABITEMS.csv and D_ITEMS.csv from mimic-iii-clinical-database-1.4
item_id = [198, 50809, 211, 456, 646, 618, 8441, 455, 676, 226730, 50820, 226512]
item_dict = {198: "glasgow coma scale", 50809: "glucose", 211: "heart rate", 456: "mean BP", 646: "oxygen saturation", 618: "respiratory rate", 8441: "diastolic BP", 455: "systolic BP", 676: "temperature", 226730: "height", 50820: "pH", 226512: "weight"}

data = pd.DataFrame()
for file in events_files:
    print(f'Processing {file}')
    events = pd.read_csv(file)
    events.drop(columns=["VALUEUOM","HADM_ID", "SUBJECT_ID"], inplace=True)
    for ICUSTAY_ID in events.ICUSTAY_ID.unique():
        events_per_ICUSTAY_ID = events[events.ICUSTAY_ID == ICUSTAY_ID]
        # Only select partial observations 
        events_per_ICUSTAY_ID = events_per_ICUSTAY_ID[events_per_ICUSTAY_ID['ITEMID'].isin(item_id)]
        # Add item name based on ITEMID
        events_per_ICUSTAY_ID['ITEM'] = events_per_ICUSTAY_ID.ITEMID.map(item_dict)
        # Replace error observations value with nan
        events_per_ICUSTAY_ID.loc[events_per_ICUSTAY_ID['VALUE'].astype(str).str.contains('error', case=False, na=False), 'VALUE'] = np.nan
        # Add columns for observations
        for item_name in list(item_dict.values()):
            events_per_ICUSTAY_ID[item_name] = np.nan
        # Set item value based on the observations name and value
        for index, value in enumerate(events_per_ICUSTAY_ID['ITEM']):
            events_per_ICUSTAY_ID.loc[events_per_ICUSTAY_ID['ITEM'].index[index], value] = events_per_ICUSTAY_ID['VALUE'][events_per_ICUSTAY_ID['ITEM'].index[index]]
        
        events_per_ICUSTAY_ID.drop(columns=["VALUE", "ITEM", "ITEMID"], inplace=True)
        events_per_ICUSTAY_ID = events_per_ICUSTAY_ID.replace(['GREATER THAN 500', '>500', 'ERROR'], [500, 500, np.nan])
        # Convert columns to numeric type
        events_per_ICUSTAY_ID.iloc[:,2:] = events_per_ICUSTAY_ID.iloc[:,2:].apply(pd.to_numeric, errors='coerce')
        # Aggregate rows based on same ICUSTAY_ID and CHARTTIME
        events_per_ICUSTAY_ID = events_per_ICUSTAY_ID.groupby(["ICUSTAY_ID", "CHARTTIME"]).agg(dict(zip(list(item_dict.values()), ['mean']*len(item_dict)))).reset_index()
        # Convert time type data
        events_per_ICUSTAY_ID['CHARTTIME'] = pd.to_datetime(events_per_ICUSTAY_ID['CHARTTIME']) 
        events_per_ICUSTAY_ID['INTIME'] = all_stays[all_stays.ICUSTAY_ID ==ICUSTAY_ID]['INTIME'].item()
        events_per_ICUSTAY_ID['INTIME'] = pd.to_datetime(events_per_ICUSTAY_ID['INTIME'])
        # Get hours, indicating the number of hours since ICU admission where the observation was made
        events_per_ICUSTAY_ID['hours'] = (events_per_ICUSTAY_ID.CHARTTIME - events_per_ICUSTAY_ID.INTIME).dt.total_seconds()/3600
        # Only keep the first 48 hours observations
        events_per_ICUSTAY_ID = events_per_ICUSTAY_ID[events_per_ICUSTAY_ID['hours']<=48]
        events_per_ICUSTAY_ID = events_per_ICUSTAY_ID[events_per_ICUSTAY_ID['hours']>=0]
        events_per_ICUSTAY_ID['day'] = (events_per_ICUSTAY_ID['hours'] // 24 + 1).astype(int)
        # Add several columns from all_stays table
        events_per_ICUSTAY_ID['mortality'] = all_stays[all_stays.ICUSTAY_ID ==ICUSTAY_ID]['MORTALITY'].item()
        events_per_ICUSTAY_ID['age'] = all_stays[all_stays.ICUSTAY_ID ==ICUSTAY_ID]['AGE'].item()
        events_per_ICUSTAY_ID['gender'] = all_stays[all_stays.ICUSTAY_ID ==ICUSTAY_ID]['GENDER'].item()
        # Encode gender column
        events_per_ICUSTAY_ID['gender'] = events_per_ICUSTAY_ID['gender'].replace(['F', 'M'], [1, 2])
        events_per_ICUSTAY_ID.drop(columns=["CHARTTIME", "INTIME"], inplace=True)
        events_per_ICUSTAY_ID = events_per_ICUSTAY_ID.set_index("ICUSTAY_ID")

        data = pd.concat([data,events_per_ICUSTAY_ID])

In [26]:
data = data.rename(columns={'ICUSTAY_ID': 'icustay'})
data = data.set_index('icustay')

In [27]:
data

,glasgow coma scale,glucose,heart rate,mean BP,oxygen saturation,respiratory rate,diastolic BP,systolic BP,temperature,height,pH,weight,hours,day,mortality,age,gender
icustay,,,,,,,,,,,,,,,,,
206884,15.0,NaN,85.0,NaN,100.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,1.643333,1,0,51.126626,M
206884,NaN,NaN,89.0,NaN,98.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,2.643333,1,0,51.126626,M
206884,NaN,NaN,95.0,NaN,100.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,3.143333,1,0,51.126626,M
206884,NaN,NaN,93.0,NaN,100.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,3.643333,1,0,51.126626,M
206884,NaN,NaN,96.0,NaN,99.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,4.643333,1,0,51.126626,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292589,NaN,NaN,93.0,NaN,100.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,42.141389,2,0,88.670773,F
292589,14.0,NaN,85.0,109.0,98.0,14.0,89.0,158.0,NaN,NaN,NaN,NaN,43.141389,2,0,88.670773,F
292589,NaN,NaN,91.0,NaN,99.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,44.141389,2,0,88.670773,F


In [28]:
data.to_csv("data.csv", )